# As raster imagery

A lot of packages rasterize HEALPix maps using a destination pull interpolation (most common: nearest neighbour).

Existing software:
- `healpy`: static image, single variable, a few projections (gnomonic, cartesian, mollweide), spherical only
- `easy.gems`: static image, single variable, projections via `cartopy`, spherical only

In [ ]:
import xdggs

ds = xdggs.tutorial.open_dataset("air_temperature", "healpix").compute().dggs.decode()
ds

## plotting with `easy.gems`

[Code](https://github.com/mpimet/easygems) | [Documentation](https://easy.gems.dkrz.de/Processing/healpix/healpix_cartopy.html) | [Cookbook](https://projectpythia.org/healpix-cookbook/notebooks/easygems/)

The `easygems` package comes with the `healpix_show` function that will first interpolate the data onto a regular grid and then use the standard regular grid visualization functionality of `matplotlib` / `cartopy`:

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cf
import easygems.healpix as egh
import matplotlib.pyplot as plt

In [ ]:
# special format needed for the cell ids coordinate / the cells dimension
var = (
    ds.assign_coords(
        crs=lambda ds: (
            (),
            0,
            {
                "grid_mapping_name": "healpix",
                "healpix_nside": 2**ds.dggs.grid_info.level,
                "healpix_nest": "nest",
            },
        )
    )
    .drop_indexes("cell_ids")
    .rename_dims({"cells": "cell"})
    .rename_vars({"cell_ids": "cell"})
    .isel(time=0)
    .set_xindex("cell")
    .get("air")
)

projection = ccrs.Robinson()
fig, ax = plt.subplots(
    figsize=(8, 4), subplot_kw={"projection": projection}, constrained_layout=True
)
ax.set_extent([200, 350, 14, 80], crs=ccrs.PlateCarree())
egh.healpix_show(var, ax=ax)
ax.add_feature(cf.COASTLINE, linewidth=0.8)
ax.add_feature(cf.BORDERS, linewidth=0.4)